In [157]:
from math import sqrt
from numpy import concatenate
from matplotlib import pyplot as plt
from pandas import read_csv
from pandas import DataFrame
from pandas import concat
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
import numpy as np
import pandas as pd
import time
from W_settings import *
import datetime
from sklearn.svm import SVR
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

import os


In [158]:
separation_date = '31/12/2010'
begin_test = '01/01/2015'
test_date = '29/11/2014'
begin_date = '01/01/2005'
final_date = '31/12/2015'

In [159]:
def get_season(dt):
    # it returnts string
    #print('type of dt:' + str(type(dt)))
    #print(dt)
    dt = dt.date()
    if (dt.month < 3):
        return 'H'
    elif (dt.month < 6):
        return 'P'
    elif (dt.month < 9):
        return "E"
    elif (dt.month < 12):
        return "A"
    else:
        return "H"


In [160]:
def prepare_data(df_temp, keep_season=False, keep_wind=False):
    df_temp = df_temp.drop(["Date"], axis=1).dropna(axis=0, how="any")

    """
    for col in salouel_data.columns:
        train_data[col] = train_data[col].apply(scale_transformer, args=(settings_array[col],))

    for col in test_data.columns:
        test_data[col] = test_data[col].apply(scale_transformer, args=(settings_array[col],))
    """
    
    #df_temp = pd.get_dummies(train_data)
    #print(test_data.info())
    print("Dataframes saved...")
    return df_temp



In [161]:
cols = ['Date', 'NO2', 'O3', 'PM10', 'RR', 'TN', 'TX', 'TM',
                     'PMERM', 'FFM', 'DXY', 'UM', 'GLOT']


In [162]:

def preprocessing_data(df):
    df = df.iloc[1:]
    df.columns = cols
    df = df.drop(["Date", 'NO2', 'O3'], axis=1)
    df = df.astype(float)
    #df = df.dropna(axis=0, how="any")
    return df


In [163]:
from pandas import read_csv
creil_raw_data = pd.read_csv("Moyennes_J_creil_2005_2015.csv", header=None, sep=';', decimal=',')
roth_raw_data = pd.read_csv("Moyennes_J_roth_2005_2015.csv", header=None, sep=';', decimal=',')
salouel_raw_data = pd.read_csv("Moyennes_J_salouel_2005_2015.csv", header=None, sep=';', decimal=',')


In [164]:
def clean_data(array_raw_data, cut):
    global begin_date
    # concat is first
    conc = True
    df = array_raw_data.iloc[1:]
    df.columns = cols
    df_temp = df.drop(["Date", 'NO2', 'O3', 'PM10', 'TN', 'TX'], axis=1)
    df_temp = df_temp.astype(float)
    df_temp["Date"] = pd.to_datetime(df["Date"])
    if cut == True:
        df_temp = df_temp[df_temp['Date'] > separation_date]
        begin_date = '01/01/2010'
    else:
        begin_date = '01/01/2005'
    # df_temp["Date"] = pd.to_datetime(df["Date"])

    return df_temp

In [176]:

df_temp = clean_data(salouel_raw_data, False)
df = prepare_data(df_temp)


Dataframes saved...


In [177]:
X_std = StandardScaler().fit_transform(df)

In [178]:
Y_pca = pca.fit_transform(X_std)


In [179]:

pca.explained_variance_ratio_

array([0.33369083, 0.22591699, 0.14435912, 0.12977589, 0.0856972 ,
       0.05835663, 0.02220334])

In [181]:
X_std.head()


AttributeError: 'numpy.ndarray' object has no attribute 'head'